<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Resolução do Exercício 4.

# Prática adicional sobre joins com pandas

Vamos trabalhar com um conjunto de dados [Coupon Redemption_SMOTE_ Feature Selection](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data) de campanha de marketing. É representado por um conjunto de tabelas relacionais:

* train: relaciona o id do consumidor, cupons de desconto, campanha de marketing e contém a variável de destino 'redemption_status', que informa se o cupom foi resgatado ou não.

* campaign_data: data de início e término da campanha e tipo de campanha

* customer_demographics: faixa etária, estado civil, tamanho da família, número de filhos, nível de renda, proprietário / inquilino / hipoteca

* coupon_item_mapping: cada cupom oferece benefícios em diferentes itens

* item_data: marca, tipo de marca e categoria do item

* customer_transaction_data: dados sobre compras feitas pelos consumidores. Não sabemos em qual compra utilizaram o cupom de desconto, talvez a promoção funcione para mais de uma compra.

<img src='img/Schema.png' align='center' width=40%/>

In [4]:
!pip install scikit-learn

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 991.0 kB/s eta 0:00:11
   - -------------------------------------- 0.4/10.6 MB 4.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.6 MB 7.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.6 MB 8.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/10.6 MB 9.2 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/10.6 MB 9.6 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/10.6 MB 9.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.6 MB 10.1 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/10.6 MB 10.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.8/10.6 MB 10.5 MB/s eta 0:00:01
   ----------------- ---------------------- 4.8/10.6 MB 10.5 MB/s eta 0:00:01
   ----------------- ---------------------- 4.8/10.6 MB 10.5 MB/s eta 0:00:01


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
train = pd.read_csv("../data/train.csv")
cust_dem = pd.read_csv("../data/customer_demographics.csv")
trans = pd.read_csv("../data/customer_transaction_data.csv")
coupon_item = pd.read_csv("../data/coupon_item_mapping.csv")
item_data = pd.read_csv("../data/item_data.csv")
campaign_data = pd.read_csv("../data/campaign_data.csv")

## Exercício 1 - Train dataframe

* Olhe para o dataframe e verifique se há valores nulos.

In [3]:
train.isna().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
dtype: int64

## Exercício 2 - Customer demographics dataframe

* Verifique se há nulos
* Olhe para os dtypes
* Impute valores nulos a partir do valor modal de cada coluna
* Traga as colunas de variáveis categóricas para o dtype 'category'

In [6]:
cust_dem.isna().sum()


customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [17]:
cust_dem.no_of_children.value_counts()

no_of_children
1     645
3+     60
2      55
Name: count, dtype: int64

In [18]:

print(cust_dem.dtypes)

imputer = SimpleImputer(strategy='most_frequent')

cust_dem = pd.DataFrame(imputer.fit_transform(cust_dem), columns=cust_dem.columns)

cust_dem_clean = (
	cust_dem
	.assign(
		age_range = lambda d: d['age_range'].astype('category'),
		marital_status = lambda d: d['marital_status'].astype('category'),
		rented = lambda d: d['rented'].astype('category'),
		family_size = lambda d: d['family_size'].astype('category'),
		no_of_children = lambda d: d['no_of_children'].astype('category'),
		income_bracket = lambda d: d['income_bracket'].astype('category')  
	)
)

cust_dem_clean

customer_id       object
age_range         object
marital_status    object
rented            object
family_size       object
no_of_children    object
income_bracket    object
dtype: object


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,1,4
1,6,46-55,Married,0,2,1,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,1,5
...,...,...,...,...,...,...,...
755,1577,36-45,Married,0,2,1,5
756,1578,46-55,Married,0,3,1,6
757,1579,46-55,Married,0,1,1,4
758,1580,26-35,Married,0,2,1,5


## Exercício 3 - Transactions dataframe

* Verifique se há nulos
* Veja os dtypes
* Remova a coluna `dates`

In [20]:
print(trans.isna().sum())

print(trans.dtypes)

trans = trans.drop(columns=['date'])

date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64
date                object
customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


## Exercício 4 - Coupon-Item dataframe
* verifique se há dados nulos

In [21]:
print(coupon_item.isna().sum())


coupon_id    0
item_id      0
dtype: int64


## Exercício 5 - Item dataframe

* Verifique se há nulos
* Converta colunas categóricas para o dtype `category`

In [23]:
print(item_data.isna().sum())

print(item_data.dtypes)

item_id       0
brand         0
brand_type    0
category      0
dtype: int64
item_id        int64
brand          int64
brand_type    object
category      object
dtype: object


In [24]:
item_data = (
	item_data
	.assign(
		brand_type = lambda d: d['brand_type'].astype('category'),
		category = lambda d: d['category'].astype('category')
	)
)

## Exercício 6 - Campaign dataframe
* Verifique se há valores nulos.
* Converta as colunas `start_date` e `end_date` para `TimeStamp`.
* Calcule o período das promoções como `start_date` - `end_date`.
* Remova `start_date` e `end_date`.

In [25]:
print(campaign_data.isna().sum())

campaign_id      0
campaign_type    0
start_date       0
end_date         0
dtype: int64


In [27]:
campaign_data = (
	campaign_data
	.assign(
		start_date = pd.to_datetime(campaign_data['start_date']),
		end_date = pd.to_datetime(campaign_data['end_date']),
	)
	.assign(
		duration = lambda d: (d['end_date'] - d['start_date']).dt.days
	)
	.drop(columns=['start_date', 'end_date'])
)

C:\Users\levyv\AppData\Local\Temp\ipykernel_21200\3588927993.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  start_date = pd.to_datetime(campaign_data['start_date']),
C:\Users\levyv\AppData\Local\Temp\ipykernel_21200\3588927993.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  end_date = pd.to_datetime(campaign_data['end_date']),


## Exercício 7 - Merge Dataframes

1. Faça um merge entre train e campaign_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

2. Combine o resultado do exercício $1$) com customer_demographics. Qual coluna fica em 'on'? Quais são os critérios em 'how'?

3. Verifique os valores nulos obtidos em $2$.

4. Imputamos os valores ausentes nas colunas demográficas. Para isso, amostramos com reposição da distribuição conjunta dos atributos demográficos observados.

5. Merge com transações médias por assunto:

* Faça um groupby por consumidor no dataframe da transação

* Adicione o valor médio às variáveis contínuas (`selling_price`, `quantity`, etc)

* merge com o DataFrame obtido em 4. Qual coluna fica em `on`? Qual critério entra em `how`?

6. Merge com transações médias por item

* a. Faça um agrupamento por item no dataframe da transação

* b. Calcule o valor médio de variáveis contínuas

* c. Faça um merge com o dataframe `coupon_item`

* d. Faça um merge de c. com item_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

* e. Faça um agrupamento do dataframe d.: agrupando por `coupon_id`. Para cada grupo, calcule a moda das variáveis categóricas 'brand','brand_type,'category'.

* f. Faça um agrupamento do dataframe d. agrupando por `coupon_id`. Para cada grupo, calcule a média das variáveis contínuas.

* g. Faça uma fusão de d. com e. e então com f. Qual coluna fica em `on`? Quais são os critérios em `how`?

7. Salve o dataframe resultante em um csv para prática de classe.

In [28]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [30]:
campaign_data.head()

,campaign_id,campaign_type,duration
0,24,Y,60
1,25,Y,32
2,20,Y,130
3,23,Y,97
4,21,Y,32


In [33]:
(
    train
    .merge(campaign_data, on='campaign_id', how='left')
	.merge(cust_dem_clean, on='customer_id', how='left')
).isna().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
duration                 0
age_range            34708
marital_status       34708
rented               34708
family_size          34708
no_of_children       34708
income_bracket       34708
dtype: int64

In [34]:
trans.head()

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,1501,26830,1,35.26,-10.69,0.0
1,1501,54253,1,53.43,-13.89,0.0
2,1501,31962,1,106.50,-14.25,0.0
3,1501,33647,1,67.32,0.00,0.0
4,1501,48199,1,71.24,-28.14,0.0
